In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
import sys

In [16]:
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master")

In [17]:
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
from PIL import Image
import os
import torch.nn as nn
import torch.nn.functional as F
import math
import albumentations as A
import time
from albumentations.pytorch import ToTensorV2
from torchvision import models
import copy
import timm





In [18]:
device=torch.device('cuda'if torch.cuda.is_available() else 'cpu')
device2=torch.device('cpu')
train_path='/kaggle/input/petfinder-pawpularity-score/train/'
test_path='/kaggle/input/petfinder-pawpularity-score/test/'
submission_path='/kaggle/input/petfinder-pawpularity-score/sample_submission.csv'


traincsv=pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")
testcsv=pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")
submission=pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
traincsv['img_path']=traincsv['Id']+'.jpg'
testcsv['img_path']=testcsv['Id']+'.jpg'

In [19]:
Image.open(train_path+traincsv.iloc[1,-1])

In [20]:
class dataini(Dataset):
    def __init__(self,rootdir,dataframe,transform=None):
        self.root_dir=rootdir
        self.transform=transform
        self.dataframe=dataframe
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self,index):
        if torch.is_tensor(index):
            index = index.tolist()
        
        img_name = self.root_dir+ self.dataframe.iloc[index,-1]
        image = Image.open(img_name)
        label = torch.tensor(self.dataframe.iloc[index, -2],dtype=torch.float32)
        
    
        
        
        if self.transform:
            imagenp=np.array(image)
            augmented = self.transform(image=imagenp)
            image = augmented['image']
            
    
        return image, label
        

In [21]:
transforms1=A.Compose([A.Resize(256, 256), 
    A.RandomCrop(224, 224),
    A.HorizontalFlip(),
    A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ToTensorV2(),
    ])

In [22]:
num_epochs=10
batch_size=128
learning_rate=0.03
gamma=0.01
step=3


In [23]:
ini_set=dataini(train_path,traincsv,transform=transforms1)
test_set=dataini(test_path,testcsv,transform=transforms1)
train_set,val_set=torch.utils.data.random_split(ini_set, [8912,1000], generator=torch.Generator().manual_seed(42))

dataload={"train":DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True),
          "val":DataLoader(dataset=val_set,batch_size=batch_size)}

data_size={"train":len(train_set),
         "val":len(val_set)}
test_loader=DataLoader(dataset=test_set,batch_size=len(testcsv))







In [24]:
trains =iter (dataload['train'])
tran1=trains.next()
image,label=tran1


In [26]:
model= timm.create_model('efficientnet_b2', pretrained=True)

In [27]:
for params in model.parameters():
    params.requires_grad=False

In [28]:
model.classifier=nn.Sequential(nn.Linear(1408,500),
                                  nn.BatchNorm1d(500),
                                  nn.Dropout(0.6),
                                  nn.ReLU(),
                                  nn.Linear(500,1))

In [29]:


model=model.to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
lossfn=nn.MSELoss()

lr_sched=torch.optim.lr_scheduler.StepLR(optimizer,step_size=step,gamma=gamma)

In [30]:

def trainer(model,num_epochs,lossfn,optimizer,scheduler):
    initime=time.time()
    g=0
    for i in range(num_epochs):
        
        for phase in ['train','val']:
            
            run_loss=0
            if phase=='train':
                model.train()
            if phase=='val':
                model.eval()
            for image,label in dataload[phase]:
                image=image.to(device)
                label=label.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    
                    output = model(image)
                    output=torch.squeeze(output)
                    loss=lossfn(output,label)
                    
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                run_loss+=loss.item()*(image.size(0))
          
            if phase=='train':
                epoch_loss=run_loss/data_size[phase]
                print("epoch:{}train loss-{}".format(i,epoch_loss)) 
                scheduler.step()
            if phase=='val':
                epoch_loss=run_loss/data_size[phase]
                print("epoch:{}val loss-{}".format(i,epoch_loss))         
            if phase == 'val' and (epoch_loss <= 0.9*g or i==0):
                g=epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
    finaltime=-(initime-time.time())
    print("time taken-{}".format(finaltime))
    model.load_state_dict(best_model_wts)
    return model
        
        

In [31]:
model=trainer(model,num_epochs,lossfn,optimizer,lr_sched)

In [43]:
torch.save(model,'model1.pth')

In [32]:
for i,(image,label) in enumerate(test_loader):
    
    image=image.to(device)
    label=label.to(device)
    outputtest=model(image).to(device2)
    
    

In [33]:
outputtest=torch.squeeze(outputtest)

In [35]:
outputtest

In [36]:
my_submission = pd.DataFrame({'Id': testcsv.Id, 'Pawpularity':outputtest.detach().numpy() })
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [37]:
my_submission